# 💼 PyKwant Tutorial: Portfolio Management

This notebook introduces the `pykwant.portfolio` module.

In **PyKwant**, a Portfolio is not a complex object with internal state. It is simply a **List of Positions**.

* **Position**: A tuple of `(Instrument, Quantity)`.

* **Portfolio**: `List[Position]`.

We provide pure functions to aggregate these positions, calculating Total NPV, Portfolio Duration, and Net Risk Exposure.

## 1. Setup and Imports

In [ ]:
import math
from datetime import date

from pykwant import dates, instruments, portfolio


# Helper to print financial values nicely
def print_money(label, amount):
    print(f"{label:<25}: {amount:>12,.2f}")

## 2. Market Environment

We define a standard market environment with a flat 4% yield curve.

In [2]:
val_date = date(2025, 1, 1)


# Flat 4% Curve
def market_curve(d: date) -> float:
    t = dates.act_365(val_date, d)
    return math.exp(-0.04 * t)

## 3. Creating the Investment Universe

We will trade three different bonds:

1. **2-Year Note** (Short Term)

2. **5-Year Note** (Medium Term)

3. **10-Year Bond** (Long Term, High Duration)

In [3]:
cal = dates.Calendar(holidays=frozenset(), weekends=(6, 7))


# Factory helper to keep code clean
def create_bond(maturity_year, coupon):
    return instruments.FixedRateBond(
        face_value=instruments.Money(100.0),
        coupon_rate=coupon,
        start_date=date(2025, 1, 1),
        maturity_date=date(maturity_year, 1, 1),
        frequency_months=12,
        day_count=dates.thirty_360,
        calendar=cal,
    )


bond_2y = create_bond(2027, 0.03)  # 3% Coupon
bond_5y = create_bond(2030, 0.04)  # 4% Coupon
bond_10y = create_bond(2035, 0.05)  # 5% Coupon

## 4. Building the Portfolio

A portfolio is just a list of `Position` objects. We can mix long (positive quantity) and short (negative quantity) positions.

In [4]:
# We construct a "Barbell" strategy:
# Long the short-end and long-end, underweight the belly (5Y).

my_portfolio = [
    portfolio.Position(bond_2y, quantity=1000.0),  # $100k Face
    portfolio.Position(bond_5y, quantity=200.0),  # $20k Face
    portfolio.Position(bond_10y, quantity=500.0),  # $50k Face
]

print(f"Portfolio created with {len(my_portfolio)} positions.")

Portfolio created with 3 positions.


## 5. Valuation (Mark-to-Market)

The `portfolio_npv` function sums the market value of all positions.

In [5]:
total_value = portfolio.portfolio_npv(my_portfolio, market_curve, val_date)

print("--- Portfolio Valuation ---")
print_money("Total NPV", total_value)

--- Portfolio Valuation ---
Total NPV                :   171,591.65


## 6. Risk Aggregation

We need to understand our exposure. The `portfolio_risk` function aggregates the individual metrics.

* **Total DV01**: How much money we lose if rates rise by 1bp across the curve.

* **Portfolio Duration**: The weighted average time to repay the capital.

In [6]:
risk_report = portfolio.portfolio_risk(my_portfolio, market_curve, val_date)

print("\n--- Risk Report ---")
print_money("Market Value", risk_report["market_value"])
print_money("Total DV01 (Risk)", risk_report["total_dv01"])
print(f"{'Portfolio Duration':<25}: {risk_report['portfolio_duration']:>12.2f} Years")

# Interpretation:
# A duration of ~5.5 years means if rates rise 1% (100bps),
# the portfolio value drops by roughly 5.5%.


--- Risk Report ---
Market Value             :   171,591.65
Total DV01 (Risk)        :       -72.48
Portfolio Duration       :         4.23 Years


## 7. Exposure Analysis (Bucketing)

Risk managers often want to see exposure grouped by maturity bucket to check for concentration risk.

In [7]:
buckets = portfolio.exposure_by_maturity_year(my_portfolio, market_curve, val_date)

print("\n--- Exposure by Maturity ---")
print(f"{'Year':<10} | {'Exposure':>15}")
print("-" * 30)

for year in sorted(buckets.keys()):
    print(f"{year:<10} | {buckets[year]:>15,.2f}")


--- Exposure by Maturity ---
Year       |        Exposure
------------------------------
2027       |       97,963.35
2030       |       19,926.03
2035       |       53,702.26


## 8. Hedging Scenario

Suppose we want to **neutralize** our interest rate risk (reduce DV01 to zero) by selling (shorting) the 10-Year Bond.

**Step 1**: **Calculate Hedge Ratio** We need to offset the current Total DV01.

In [8]:
import pykwant.risk as risk_mod

# 1. Get Risk of the Hedging Instrument (1 unit of 10Y Bond)
hedge_metrics = risk_mod.calculate_risk_metrics(bond_10y, market_curve, val_date)
hedge_dv01 = hedge_metrics["dv01"]

current_dv01 = risk_report["total_dv01"]

# 2. Calculate Quantity to trade
# We want: Current_DV01 + (Qty * Hedge_DV01) = 0
# Qty = -Current_DV01 / Hedge_DV01
hedge_qty = -current_dv01 / hedge_dv01

print("\n--- Hedging Strategy ---")
print(f"Current Portfolio DV01: {current_dv01:.2f}")
print(f"10Y Bond DV01 (per unit): {hedge_dv01:.4f}")
print(f"Required Hedge Quantity: {hedge_qty:.2f} units (Short)")


--- Hedging Strategy ---
Current Portfolio DV01: -72.48
10Y Bond DV01 (per unit): -0.0879
Required Hedge Quantity: -824.54 units (Short)


**Step 2**: **Execute Hedge and Re-Check**

In [9]:
# Add the hedge position to the portfolio
hedged_portfolio = my_portfolio + [portfolio.Position(bond_10y, hedge_qty)]

# Recalculate Risk
new_risk = portfolio.portfolio_risk(hedged_portfolio, market_curve, val_date)

print("\n--- Post-Hedge Risk ---")
print_money("Original DV01", current_dv01)
print_money("New DV01", new_risk["total_dv01"])  # Should be ~0
print(f"New Duration: {new_risk['portfolio_duration']:.4f} Years (Target: 0)")


--- Post-Hedge Risk ---
Original DV01            :       -72.48
New DV01                 :         0.00
New Duration: -0.0011 Years (Target: 0)
